# 08 - Simulation Engine

## Agentic Logistics Control System

This notebook demonstrates the simulation engine for logistics modeling:
- Route simulation with traffic
- Rerouting simulation
- Reassignment simulation
- ETA prediction

### Control Loop Role: PLANNING Engine

In [ ]:
import sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from src.models import Truck, TruckStatus, Location, Load, LoadPriority, TrafficCondition, TrafficLevel
from src.planning.simulation_engine import SimulationEngine

engine = SimulationEngine()
print("Simulation engine loaded!")

In [ ]:
# Create test data
origin = Location(latitude=40.7128, longitude=-74.0060, name="NYC Warehouse")
destination = Location(latitude=40.7580, longitude=-73.9855, name="Midtown Customer")

truck = Truck(id="TRK-001", name="Alpha", status=TruckStatus.EN_ROUTE,
              current_location=origin, fuel_level_percent=75)

load = Load(id="LOAD-001", description="Electronics", weight_kg=2000,
            priority=LoadPriority.HIGH, pickup_location=origin, delivery_location=destination)

traffic = [TrafficCondition(segment_id="SEG-1", level=TrafficLevel.HEAVY, speed_kmh=25)]

In [ ]:
# Simulate route
result = engine.simulate_route(origin, destination, traffic, truck, load)
print("Route Simulation:")
for k, v in result.items():
    if not isinstance(v, dict):
        print(f"  {k}: {v}")

In [ ]:
# Simulate rerouting
alt_routes = [
    {"name": "Route 9A", "distance_km": 12, "traffic_factor": 1.1},
    {"name": "Local Streets", "distance_km": 8, "traffic_factor": 1.3},
]

reroute_results = engine.simulate_reroute(truck, origin, destination, traffic, alt_routes)
print("\nReroute Comparison:")
for r in reroute_results:
    savings = r.get('time_savings_minutes', 0)
    print(f"  {r['route_name']}: {r['time_minutes']:.1f} min, ${r['total_cost']:.2f}" + 
          (f" (saves {savings:.1f} min)" if savings else ""))

In [ ]:
# Predict ETA
eta = engine.predict_eta(truck, destination, traffic)
print("\nETA Prediction:")
print(f"  ETA: {eta['eta']}")
print(f"  Minutes from now: {eta['eta_minutes_from_now']}")
print(f"  Confidence: {eta['confidence']}")

## Next: 09_scenario_generator.ipynb